In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=1)

In [ ]:
# get data
from eden.converter.graph.gspan import gspan_to_eden
from itertools import islice
def get_graphs(dataset_fname, size=100):
    return  islice(gspan_to_eden(dataset_fname),size)

#dataset_names = !cat NCI60/names
#dataset = dataset_names[3]
#dataset_fname = 'NCI60/' + dataset + '_orig_pos.gspan'
#print 'Working with dataset: %s' % dataset 

dataset_fname = 'bursi.pos.gspan'

In [ ]:
'''
first we test the core/interface extractor, since it is most essential
'''
from graphlearn.utils.draw import  cip_to_drawable_graph, graphlearn_draw
from graphlearn.graphtools import extract_core_and_interface

from eden.graph import Vectorizer
vectorizer=Vectorizer()

#get a graph
graphs = get_graphs(dataset_fname)
graph = next(graphs)

graphlearn_draw(graph, size=8, font_size=8, secondary_vertex_label='ID',
        colormap='Paired', invert_colormap=False,node_border=0.5, vertex_color='_labels_',
        vertex_alpha=0.7, node_size=900)

radius_list=[0,2]
thickness_list=[2,4]
cip=extract_core_and_interface(1,graph,radius_list=radius_list,thickness_list=thickness_list,vectorizer=vectorizer)
graph=cip_to_drawable_graph(cip)

graphlearn_draw(graph,
                          n_graphs_per_line=7, size=5, 
                          colormap='Paired', invert_colormap=False,node_border=1,
                          vertex_alpha=0.6, edge_alpha=0.2, node_size=450)

In [ ]:
'''
demonstrating more extract core
'''
from eden.converter.graph.gspan import gspan_to_eden
from graphlearn.utils.draw import graphlearn_draw, cip_to_drawable_graph
from graphlearn.graphtools import extract_core_and_interface
from eden.graph import Vectorizer

graphs = get_graphs(dataset_fname, size=5)
for graph in graphs:
    graphs_list=[]
    for node in range(1,4):
        cips = extract_core_and_interface(node,graph,radius_list=[2],thickness_list=[1],vectorizer=Vectorizer())
        # draw cip on the original graph and save in nice_graphs
        cip_graphs = cip_to_drawable_graph(cips)
        for g,c in zip(cip_graphs,cips):
            g.graph['info'] = str(c.interface_hash)
        graphs_list += cip_graphs
    # draw marked graphs
    graphlearn_draw(graphs_list, contract=False,
                              n_graphs_per_line=5, size=7, colormap='Paired', invert_colormap=False,node_border=1,
                              vertex_alpha=0.6, edge_alpha=0.2, node_size=450)

In [ ]:
%%time
'''
induce a grammar and train an estimator
'''
from graphlearn.graphlearn import GraphLearnSampler
training_graphs = get_graphs(dataset_fname, size=200)
sampler=GraphLearnSampler(radius_list=[0,1],thickness_list=[2],
                          min_cip_count=2, min_interface_count=2)
sampler.fit(training_graphs,n_jobs=-1,batch_size=10)

print('graph grammar stats:')
interface_counts, core_counts, cip_counts = sampler.grammar().size()
print('#interfaces: %d   #cores: %d   #core-interface-pairs: %d' % (interface_counts, core_counts, cip_counts))

In [ ]:
#draw production rules
from graphlearn.utils.draw import draw_grammar
draw_grammar(sampler.grammar().productions, contract=True,
             n_productions=6,n_graphs_per_line=7, 
             size=3, colormap='Paired', node_border=1, vertex_alpha=0.7, edge_alpha=0.2, node_size=350)

In [ ]:
#grammar statistics
from graphlearn.utils.draw import draw_grammar_stats
draw_grammar_stats(sampler.grammar().productions, size=(10,5))

In [ ]:
'''
Demonstrating the filter function
'''
#get a graph
graphs = get_graphs(dataset_fname, size=1)
graph = next(graphs)
from eden.util import display
print 'We work on this graph'
display.draw_graph(graph, size=10, secondary_vertex_label='ID',
                   colormap='Paired', invert_colormap=False,node_border=0.5, vertex_color='_labels_',
                   vertex_alpha=0.5, node_size=650)

from graphlearn.utils.draw import cip_to_drawable_graph, graphlearn_draw
def mark_and_draw(cip):
    graph=cip_to_drawable_graph(cip)
    graphlearn_draw(graph, n_graphs_per_line=7, size=5,
                              colormap='Paired', invert_colormap=False,node_border=1,
                              vertex_alpha=0.6, edge_alpha=0.2, node_size=450)
    
radius_list=[0,1,2]
thickness_list=[1,2]
graph.node[1]['no_root']=True
#lets see.. without filter:
from graphlearn.graphtools import extract_core_and_interface, filter
cip=extract_core_and_interface(1,graph,radius_list=radius_list,thickness_list=thickness_list,vectorizer=vectorizer)
print 'no filter'
mark_and_draw(cip)


# lets do the same but with the filter
cip=extract_core_and_interface(1,graph,filter=filter,radius_list=radius_list,thickness_list=thickness_list,vectorizer=vectorizer)
print 'with filter: nothing should appear,,,'
mark_and_draw(cip)

# filter is ignoring unfiltered things:
cip=extract_core_and_interface(2,graph,filter=filter,radius_list=radius_list,thickness_list=thickness_list,vectorizer=vectorizer)
print 'with filter but we chose node 2 as root'
mark_and_draw(cip)

# no core attribute
graph.node[1]["not_in_core"]=True
graph.node[1].pop('no_root')
cip=extract_core_and_interface(2,graph,filter=filter,radius_list=radius_list,thickness_list=thickness_list,vectorizer=vectorizer)
print 'with filter but node "1" cant be part of a core, we gave 0 and 2 as radius list and only radius 2 will show up'
mark_and_draw(cip)

.